# LangChain Academy Workshop

Welcome to LangChain Academy Workshop!

## Context

LangChain aims to make it easy to build LLM applications. One type of LLM application you can build is an agent. There’s a lot of excitement around building agents because they can automate a wide range of tasks that were previously impossible.

In practice though, it is incredibly difficult to build systems that reliably execute on these tasks. As we work with  to put agents into production, we’ve learned that more control is often necessary. You might need an agent to always call a specific tool first or use different prompts based on its state. 

To tackle this problem, some smart folks built [LangChain](https://www.langchain.com/) & [LangGraph](https://langchain-ai.github.io/langgraph/) — frameworks for building smart GenAI systems, RAGs and agents applications. 

## Chat models

In this workshop, we'll be using [Chat Models](https://python.langchain.com/v0.2/docs/concepts/#chat-models), which do a few things take a sequence of messages as inputs and return chat messages as outputs. LangChain does not host any Chat Models, rather we rely on third party integrations. [Here](https://python.langchain.com/v0.2/docs/integrations/chat/) is a list of 3rd party chat model integrations within LangChain! By default, the workshop will use [ChatGoogleGenerativeAI](https://python.langchain.com/docs/integrations/chat/google_generative_ai/) because it is both popular and performant and free. As noted, please ensure that you have an `GOOGLE_API_KEY`.

Let's check that your `GOOGLE_API_KEY` is set and, if not, you will be asked to enter it.

In [5]:
%%capture --no-stderr
%pip install --quiet -U langchain-google-genai langchain_core langchain_community

In [6]:
import getpass
import os

if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter your Google AI API key: ")

The model we will use is Gemini 2.5 Flash or Gemini 2.5 Pro, which is a chat model that can handle a wide range of tasks, including answering questions, generating text, and more.

There are [a few standard parameters](https://python.langchain.com/v0.2/docs/concepts/#chat-models) that we can set with chat models. Two of the most common are:

* `model`: the name of the model
* `temperature`: the sampling temperature

`Temperature` controls the randomness or creativity of the model's output where low temperature (close to 0) is more deterministic and focused outputs. This is good for tasks requiring accuracy or factual responses. High temperature (close to 1) is good for creative tasks or generating varied responses. 

In [7]:
from langchain_google_genai import ChatGoogleGenerativeAI
gemini_flash_chat = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0)
gemini_pro_chat = ChatGoogleGenerativeAI(model="gemini-2.5-pro", temperature=0)

Chat models in LangChain have a number of [default methods](https://python.langchain.com/v0.2/docs/concepts/#runnable-interface). For the most part, we'll be using:

* `stream`: stream back chunks of the response
* `invoke`: call the chain on an input

And, as mentioned, chat models take [messages](https://python.langchain.com/v0.2/docs/concepts/#messages) as input. Messages have a role (that describes who is saying the message) and a content property. We'll be talking a lot more about this later, but here let's just show the basics.

In [8]:
from langchain_core.messages import HumanMessage

# Create a message
msg = HumanMessage(content="Hello world", name="Lance")

# Message list
messages = [msg]

# Invoke the model with a list of messages 
gemini_flash_chat.invoke(messages)

AIMessage(content='Hello there!', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': []}, id='run--c41294f9-f153-4a4e-8e76-b553fb016b7f-0', usage_metadata={'input_tokens': 3, 'output_tokens': 3, 'total_tokens': 39, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 33}})

We get an `AIMessage` response. Also, note that we can just invoke a chat model with a string. When a string is passed in as input, it is converted to a `HumanMessage` and then passed to the underlying model.


In [9]:
gemini_flash_chat.invoke("hello world")

AIMessage(content='Hello there! World says hi back. 😊', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': []}, id='run--13153a78-3d5e-4100-a1d9-fd8e229244a5-0', usage_metadata={'input_tokens': 3, 'output_tokens': 9, 'total_tokens': 66, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 54}})

In [10]:
gemini_pro_chat.invoke("hello world")

AIMessage(content='Hello there! The classic "hello world."\n\nHow can I help you today?', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-pro', 'safety_ratings': []}, id='run--d7194025-5a72-4db4-a282-220e38336bd3-0', usage_metadata={'input_tokens': 3, 'output_tokens': 17, 'total_tokens': 1125, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 1105}})

The interface is consistent across all chat models and models are typically initialized once at the start up each notebooks. 

So, you can easily switch between models without changing the downstream code if you have strong preference for another provider.
